This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

# Load the results + get generated suffixes

In [6]:
results_acc_dir = oj('/home/chansingh/interpretable-autoprompting/results/generalization_acc/accs_sweep')

In [15]:
accs = [
    pd.DataFrame.from_dict(pkl.load(open(oj(results_acc_dir, d), 'rb')))
    for d in os.listdir(results_acc_dir)
]


In [18]:
df = pd.concat(accs)
df['max_digit'] = 10 # PLACEHOLDER!!!

In [ ]:
math_one = ['square_one', 'exp_one', 'double_one', 'fibonacci_one']
math_two = ['add_two', 'multiply_two', 'divide_two', 'subtract_two',
                       'max_two', 'first_two']
math_three = ['add_three',
                         'multiply_three', 'max_three', 'first_three']

In [ ]:
row1 = df[(df.checkpoint == 'EleutherAI/gpt-neo-2.7B') & (df.max_digit == 10) & (df.task_names.isin(math_one + math_two))]
print('Chandan, did you fixe the placheolder?')

In [25]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df.head(100))

,checkpoint,prompt,task_name,n_shots,prompt_actual,acc
0,EleutherAI/gpt-neo-2.7B,autoprompt,add_two,1,addedthe +the use worked,8.000000
1,EleutherAI/gpt-neo-2.7B,iprompt,add_two,1,Write a function int add(,1.733333
2,EleutherAI/gpt-neo-2.7B,,add_two,1,,1.466667
3,EleutherAI/gpt-neo-2.7B,manual,add_two,1,Return the sum of the inputs.,18.800000
4,EleutherAI/gpt-neo-2.7B,autoprompt,multiply_two,1,multiplythethe �the Multiple,0.133333
5,EleutherAI/gpt-neo-2.7B,iprompt,multiply_two,1,When you multiply two (,3.466667
6,EleutherAI/gpt-neo-2.7B,,multiply_two,1,,0.800000
7,EleutherAI/gpt-neo-2.7B,manual,multiply_two,1,Return the product of the inputs.,0.800000
0,EleutherAI/gpt-j-6B,autoprompt,square_one,1,multiplythe hypot Norttheirl,71.428571
1,EleutherAI/gpt-j-6B,iprompt,square_one,1,Write a function that calculates square,85.714286


In [ ]:
df